In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator


payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:%}"
stk='MMM'

from ta.trend import EMAIndicator

#df = yf.download(stk, start_date, end_date,interval=intr).dropna()

df = yf.download(stk, start_date, end_date,interval='1d').dropna()

df['Close1']=df.Close.shift(1)
df['ema_d']=EMAIndicator(close=df.Close1,window=14).ema_indicator()
df['rsi']=RSIIndicator(close=df.Close1).rsi()
#print(df.head)
df=df.dropna()
thr99=90
thr1=10

print(thr1,thr99)

# Our trading condition:
df['Long'] = np.where(df.rsi < thr1, 1, 0)
df['Short']=np.where(df.rsi > thr99, -1, 0)
df['Position']=df.Long+df.Short



# The returns of the Buy and Hold strategy:
df['Hold'] = np.log(df['Close'] / df['Close1'])
#print(df)
df['day']= np.log(df['Close'] / df['Open'])
#hold_prof=df.iloc[0,0]

# The returns of the Moving Average strategy:
df['Strategy'] = df['Position'] * np.log(df['Close'] / df['Open'])

df['Strategy'] = np.where(df.Position ==df.Position.shift(1),
                          (df['Position'] * np.log(df['Close'] / df['Close1'])),
                          (df['Position'] * np.log(df['Close'] / df['Open'])))


# We need to get rid of the NaN generated in the first row:
df.dropna(inplace=True)

print(df.tail())
returns = np.exp(df[['Hold', 'Strategy']].sum()) - 1
print(f"Buy and hold return: {round(returns['Hold']*100,2)}%")
print(f"Strategy return: {round(returns['Strategy']*100,2)}%")

n_days = len(df)
# Assuming 252 trading days in a year:
ann_returns = 252 / n_days * returns
print(f"Buy and hold annualized return: {round(ann_returns['Hold']*100,2)}%")
print(f"Strategy annualized return:{round(ann_returns['Strategy']*100,2)}%")

df.to_csv('bt.csv')

[*********************100%***********************]  1 of 1 completed
10 90
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-09-24  181.500000  182.070007  180.619995  181.039993  179.578552   
2021-09-27  180.830002  183.160004  180.809998  181.789993  180.322510   
2021-09-28  181.820007  182.850006  179.770004  180.050003  178.596558   
2021-09-29  179.919998  181.389999  179.169998  180.880005  179.419861   
2021-09-30  181.720001  182.059998  175.369995  175.419998  174.003937   

             Volume      Close1       ema_d        rsi  Long  Short  Position  \
Date                                                                            
2021-09-24  2076900  181.529999  184.060040  35.365265     0      0         0   
2021-09-27  2052100  181.039993  183.657367  34.583391     0      0         0   
2021-09-28  2290400  181.789993  183.408384  36.883498     0      0         0   
2